In [ ]:
import numpy as np 
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import sklearn as sk

# Step 1: Ticker
TICKER = "SPY"

# Step 2: Price Behavior 
INTERVAL = "1h"
if INTERVAL == "1h":
    PERIOD = "730d"
else:
    PERIOD = "max"

# Step 3: Price difference
SHIFT = 1

# Step 5: Data Subsection
LOOKBACK = 10000

def get_data(ticker = TICKER, interval = INTERVAL, lookback = LOOKBACK):

    # Step 1: Download Data
    df = yf.download(ticker, interval = interval, period = PERIOD)

    # Step 2: Select only Level 0 of Multilevel Columns
    df.columns = df.columns.get_level_values(0)

    # Step 3: Simplifying df for Plots
    df = df.reset_index(drop=True)

    return df.iloc[-lookback:, :]
        # -lookback: all loockback rows
        # : all columns 

# Defining y/the Dependent/the Target Variable
# Lets inspect how a tech Indicator is related to how a price changes tomorrow
# For eg. with the RSI, you want to have a negative correlation
def add_target(df, shift = SHIFT):


def main():
    df = get_data()

    return df

df = main()
df

/var/folders/jj/9j0c_6196zx24tx5jb3ngnym0000gn/T/ipykernel_11202/3323275761.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, interval = interval, period = PERIOD)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
0,381.470093,381.649994,377.850006,379.230011,14574668
1,379.179993,382.140015,378.850006,381.480011,10793846
2,380.670013,381.290009,379.089996,379.170013,8601333
3,381.500000,381.684998,380.010010,380.640015,6971623
4,381.359985,382.230011,381.059998,381.510010,6227816
...,...,...,...,...,...
5081,672.310120,673.000000,670.372009,670.479980,6729432
5082,668.710022,672.590027,668.219971,672.330017,7123778
5083,665.839478,669.909973,664.969971,668.700012,10600848
5084,663.739990,666.141479,662.169983,665.830017,15165730
